In [ ]:
!gdown  18cij6foP2ilw2NtadINSnYP6UUbenXCf

Downloading...
From: https://drive.google.com/uc?id=18cij6foP2ilw2NtadINSnYP6UUbenXCf
To: /content/scc5966.zip
100% 18.4M/18.4M [00:00<00:00, 66.0MB/s]


In [ ]:
!unzip scc5966.zip

Archive:  scc5966.zip
  inflating: movie_reviews.csv       
  inflating: movies_data.csv         
  inflating: test_data.csv           
  inflating: train_data .csv         
  inflating: users_data.csv          


In [ ]:
import pandas as pd 

df_movie_reviews = pd.read_csv('movie_reviews.csv')

df_movies_data = pd.read_csv('movies_data.csv')

df_test_data = pd.read_csv('test_data.csv')

df_train_data = pd.read_csv('train_data .csv')

df_users_data = pd.read_csv('users_data.csv')


In [ ]:
import pandas as pd
from tqdm import tqdm 

df=pd.DataFrame(index = df_train_data['user_id'].unique(), columns=df_train_data['movie_id'].unique())

for movie in tqdm(df_train_data['movie_id'].unique()):
  temp = df_train_data[df_train_data.movie_id == movie]
  df.loc[temp.user_id, movie] = temp.rating.to_list()

100%|██████████| 3562/3562 [00:19<00:00, 179.80it/s]


In [ ]:
from sklearn.decomposition import TruncatedSVD as SVD
import numpy as np

In [ ]:
new_df_0 = df.fillna(0)

In [ ]:
global_avg = np.nanmean(df.values.tolist())

In [ ]:
global_avg

In [ ]:
new_df_globalAVG = df.fillna(global_avg)

In [ ]:
items_avg = np.mean(df)

In [ ]:
items_avg

In [ ]:
new_df_ItemsAVG = df.fillna(value = items_avg.to_dict())

In [ ]:
user_avg = np.mean(df.T)

In [ ]:
new_df_UserAVG = df.T.fillna(value = user_avg.to_dict())

In [ ]:
new_df_UserAVG = new_df_UserAVG.T

In [ ]:
new_df_UserAVG

In [ ]:
dic_dfs = {
    'input0' : new_df_0,
    'inputGlobalMean' : new_df_globalAVG,
    'inputItemMean' : new_df_ItemsAVG,
    'inputUserMean' : new_df_UserAVG
}

In [ ]:
import random

input_type = 'inputGlobalMean'

new_df = dic_dfs[input_type]

factors 100

svd = SVD(n_components=factors, random_state=42)
data_reducted = svd.fit_transform(new_df)

data_original_svd = svd.inverse_transform(data_reducted)

df_original_svd = pd.DataFrame(data_original_svd, index=df.index, columns=df.columns)

l_mean = []
for item,row in tqdm(df_test_data.iterrows()):
  user = row['user_id']
  item = row['movie_id']
  if user not in df_train_data['user_id'].unique() and item in df_train_data['movie_id'].unique():
    l_mean.append(items_avg[item])
  elif user in df_train_data['user_id'].unique() and item not in df_train_data['movie_id'].unique():
    l_mean.append(user_avg[user])
  elif user not in df_train_data['user_id'].unique() and item not in df_train_data['movie_id'].unique():
    l_mean.append(global_avg)
  else:
    l_mean.append(df_original_svd.loc[user][item])

df_test_data['rating'] = l_mean
df_rating = df_test_data[['id','rating']]
df_rating = df_rating.set_index('id')
name = 'FC_SVD_AL_' + input_type + '_' + str(factors) + '.csv'
df_rating.to_csv('/content/drive/MyDrive/USP/Doctorate/Credits/Recomender_Systems/results-kaggle/' +name)